### auxfunc.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains all the auxiliary functions used in the other .ipynb files of the repository

In [1]:
# IMPORTS
import import_ipynb
import constants
import pandas as pd
import numpy as np
import os

importing Jupyter notebook from constants.ipynb


In [2]:
def get_anchors_name(directory_path):
    '''
    Examine the dataset at 'directory_path' in order to get the anchor names.

    Parameters:
    directory_path (str): The path of the directory which contains the dataset.

    Returns:
    anchors (list): Returns a list which values are the anchors' name.
    '''

    anchors = []
    
    for path in os.listdir(directory_path):
        if path.lower().endswith(".csv"):
            anchors.append(path)

    anchors.sort()

    return anchors

# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
# https://stackoverflow.com/questions/5899497/how-can-i-check-the-extension-of-a-file

In [3]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (list): a list which values are the anchors' dataframe.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in range(len(dataframes)):
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)

In [4]:
def calculate_burst_quantity(length):
    """
    Calculates the amount of bursts over all dataframes.

    Parameters:
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    bursts (int): An integer which value represents the amount of bursts / configurations.
    """

    bursts = length // constants.BURST_SIZE
    end = constants.BURST_SIZE * bursts

    if (end < length):
        bursts = bursts + 1
    
    return bursts

In [5]:
def calculate_average_RSS(scenario_dataframe, length):
    """
    Calculates the average RSS value over a burst of 'constants.BURST_SIZE' consecutive measurements.
    This is the implementation of (3) on the reference paper.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    average_RSS (list): Returns a list of average RSS values.
    """
    
    average_RSS = []
    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    bursts = calculate_burst_quantity(length)

    start = 0
    for i in range(bursts):
        end = start + constants.BURST_SIZE
        # print("start: ", start)
        if (end <= length):
            # print("end: ", end)
            average_RSS.append(round(np.mean(power_values[start:end]), 3))
        else:
            # print("end: ", length)
            average_RSS.append(round(np.mean(power_values[start:length]), 3))
        start = end

    return average_RSS